In [3]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

Reading the datasets that we have. We will need movies and the ratings files. 

In [4]:
movie = pd.read_csv('movie.csv')
rating= pd.read_csv('rating.csv')

Quick view of the two dataframes, the rows and the columns

In [3]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


rating.head()

Using pandas' replace function, we remove the year from the title in the movie dataframe and add a separate year column.
This is done using regular expressions. Find a year stored between parentheses. We specify the parantheses so we don't conflict with movies that have years in their titles.

First we create the new Year column in the movie dataframe.

In [5]:
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movie['year'] = movie.year.str.extract('(\d\d\d\d)',expand=False)

Now removing the year from the title in the Title column. 

In [6]:
#Removing the years from the 'title' column
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movie['title'] = movie['title'].apply(lambda x: x.strip())

C:\Users\Dell\AppData\Local\Temp/ipykernel_1832/2563781091.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')


In [7]:
movie.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Collaborative filtering doesn't recommend based on the features of the movie. The recommendation is based on the likes and dislikes or ratings of the neighbours or other users. 
So we will drop the genre column, since there is no use of it. 

In [8]:
movie.drop(columns=['genres'], inplace=True)

In [9]:
movie.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


Now, coming to the ratings dataframe, we have the movieId column that is common with the movie dataframe. Each user has given multiple ratings for different movies. The column Timestamp is not needed for the recommendation system. So we can drop it. 

In [10]:
rating.drop(columns=['timestamp'],inplace=True)

In [11]:
rating.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


The technique is called Collaborative Filtering, which is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

In [12]:
user = [
            {'title':'Breakfast Club, The', 'rating':4},
            {'title':'Toy Story', 'rating':2.5},
            {'title':'Jumanji', 'rating':3},
            {'title':"Pulp Fiction", 'rating':4.5},
            {'title':'Akira', 'rating':5}
         ] 
inputMovie = pd.DataFrame(user)
inputMovie

,title,rating
0,"Breakfast Club, The",4.0
1,Toy Story,2.5
2,Jumanji,3.0
3,Pulp Fiction,4.5
4,Akira,5.0


We need to now add the movieId column from the movie dataframe into the inputMovie Dataframe. 
First filter out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [13]:
#Filtering out the movies by title
Id = movie[movie['title'].isin(inputMovie['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovie = pd.merge(Id, inputMovie)
#Dropping information we won't use from the input dataframe
inputMovie = inputMovie.drop('year', 1) #0 for rows, 1 for columns
inputMovie

C:\Users\Dell\AppData\Local\Temp/ipykernel_1832/397080606.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  inputMovie = inputMovie.drop('year', 1) #0 for rows, 1 for columns


,movieId,title,rating
0,1,Toy Story,2.5
1,2,Jumanji,3.0
2,296,Pulp Fiction,4.5
3,1274,Akira,5.0
4,1968,"Breakfast Club, The",4.0


**Finding the users who have seen the same movies from the rating dataframe**
With the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [14]:
#Filtering out users that have watched movies that the input has watched and storing it
users = rating[rating['movieId'].isin(inputMovie['movieId'].tolist())]
users.head()

,userId,movieId,rating
0,1,2,3.5
11,1,296,4.0
236,3,1,4.0
451,5,2,3.0
517,6,1,5.0


In [15]:
users.shape

(168730, 3)

In [16]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = users.groupby(['userId'])

In [17]:
#showing one such group example by getting all the users of a particular userId
userSubsetGroup.get_group(1130)

,userId,movieId,rating
166633,1130,1968,4.0


In [18]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [19]:
userSubsetGroup[0:3]

[(91,
        userId  movieId  rating
  9621      91        1     4.0
  9622      91        2     3.5
  9669      91      296     3.5
  9826      91     1274     2.5
  9903      91     1968     4.0),
 (294,
         userId  movieId  rating
  37452     294        1     4.5
  37453     294        2     4.5
  37504     294      296     4.5
  37648     294     1274     4.5
  37731     294     1968     5.0),
 (586,
         userId  movieId  rating
  81164     586        1     2.5
  81165     586        2     3.0
  81226     586      296     5.0
  81390     586     1274     4.0
  81499     586     1968     3.0)]

**Similarity of users to input user**
Next, we are going to compare all users  to our specified user and find the one that is most similar.
we're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation") 

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [20]:
userSubsetGroup = userSubsetGroup[0:100]
userSubsetGroup

[(91,
        userId  movieId  rating
  9621      91        1     4.0
  9622      91        2     3.5
  9669      91      296     3.5
  9826      91     1274     2.5
  9903      91     1968     4.0),
 (294,
         userId  movieId  rating
  37452     294        1     4.5
  37453     294        2     4.5
  37504     294      296     4.5
  37648     294     1274     4.5
  37731     294     1968     5.0),
 (586,
         userId  movieId  rating
  81164     586        1     2.5
  81165     586        2     3.0
  81226     586      296     5.0
  81390     586     1274     4.0
  81499     586     1968     3.0),
 (648,
         userId  movieId  rating
  92885     648        1     4.0
  92886     648        2     2.0
  92937     648      296     5.0
  93124     648     1274     4.0
  93237     648     1968     3.0),
 (775,
          userId  movieId  rating
  113566     775        1     4.5
  113567     775        2     2.0
  113695     775      296     5.0
  114057     775     1274     4.5
  

In [21]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovie = inputMovie.sort_values(by='movieId')
    #Get the N for the formula
    n = len(group)
    #Get the review scores for the movies that they both have in common
    temp = inputMovie[inputMovie['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp['rating'].tolist()
    #put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0


In [22]:
pearsonCorDict.items()

dict_items([(91, -0.6890618270883883), (294, 0.10783277320343156), (586, 0.7836445860269199), (648, 0.444102681159703), (775, 0.46266531814837414), (812, -0.12945217625467967), (869, 0.07624928516630236), (903, 0.0660338179744212), (1200, 0.2494610901255917), (1244, 0.29654012630945475), (1715, 0.6309898162000303), (1748, 0.5114957546028552), (1763, 0.1760901812651271), (1810, 0.6990252954195334), (1813, 0.36589645615870564), (1849, 0.06603381797442423), (1864, 0.5114957546028552), (1942, 0.23262521394079627), (1984, -0.7994259492812168), (2047, 0.5477103564747346), (2099, -0.10783277320343156), (2367, -0.10783277320343994), (2397, 0), (2515, 0.9244734516419062), (2661, 0.835703992326648), (2757, 0.8439249387982215), (2959, 0.23055616708169688), (2988, 0.29809064964264287), (3179, 0.0), (3218, 0.26413527189768793), (3268, 0.7781270639007126), (3269, 0.3606167767094639), (3318, 0.3026049692947228), (3397, 0.46107317554294897), (3487, -0.3774147062120338), (3576, 0.39620290784652895), (3

In [23]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
print(pearsonDF)
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

              0
91    -0.689062
294    0.107833
586    0.783645
648    0.444103
775    0.462665
...         ...
10621 -0.129452
10670  0.616316
10821  0.221918
10822  0.022244
10989  0.057639

[100 rows x 1 columns]


,similarityIndex,userId
0,-0.689062,91
1,0.107833,294
2,0.783645,586
3,0.444103,648
4,0.462665,775


In [24]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head(20)

,similarityIndex,userId
77,0.964486,9305
80,0.951044,9650
63,0.940064,8032
81,0.924473,9772
23,0.924473,2515
78,0.924473,9410
64,0.885937,8082
25,0.843925,2757
51,0.836428,6057
24,0.835704,2661


**Rating of selected users to all movies**
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [25]:
topUsersRating=topUsers.merge(rating, left_on='userId', right_on='userId', how='inner')
topUsersRating.head(15)

,similarityIndex,userId,movieId,rating
0,0.964486,9305,1,1.0
1,0.964486,9305,2,1.0
2,0.964486,9305,11,1.0
3,0.964486,9305,21,1.0
4,0.964486,9305,32,2.0
5,0.964486,9305,39,2.0
6,0.964486,9305,46,1.0
7,0.964486,9305,47,4.0
8,0.964486,9305,58,3.0
9,0.964486,9305,60,1.0


multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

In [26]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.964486,9305,1,1.0,0.964486
1,0.964486,9305,2,1.0,0.964486
2,0.964486,9305,11,1.0,0.964486
3,0.964486,9305,21,1.0,0.964486
4,0.964486,9305,32,2.0,1.928971


In [27]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,30.998297,105.356196
2,30.998297,85.222713
3,9.567244,21.072345
4,0.888910,2.289317
5,5.368970,13.468631


In [28]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.398774,1
2,2.749271,2
3,2.202551,3
4,2.575419,4
5,2.508606,5


In [35]:
recommendation_df.values.head(5)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [30]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
4422,5.0,4422
25744,5.0,25744
42217,5.0,42217
2068,5.0,2068
1419,5.0,1419
8785,5.0,8785
93164,5.0,93164
26587,5.0,26587
1585,5.0,1585


In [31]:
movie.loc[movie['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
1385,1419,Walkabout,1971
1533,1585,Love Serenade,1996
1984,2068,Fanny and Alexander (Fanny och Alexander),1982
4327,4422,Cries and Whispers (Viskningar och rop),1972
5311,5408,Nora,2000
8102,8785,Early Summer (Bakushû),1951
8345,25744,Haxan: Witchcraft Through the Ages (a.k.a. The...,1922
8937,26587,"Decalogue, The (Dekalog)",1989
10709,42217,Late Spring (Banshun),1949
18706,93164,Sleep Tight (Mientras duermes),2011
